<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Импорт-библиотек" data-toc-modified-id="Импорт-библиотек-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт библиотек</a></span></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Очистка-и-лемматизация" data-toc-modified-id="Очистка-и-лемматизация-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Очистка и лемматизация</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Баланс-классов" data-toc-modified-id="Баланс-классов-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Баланс классов</a></span></li><li><span><a href="#Признаки-и-целевой-признак" data-toc-modified-id="Признаки-и-целевой-признак-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Признаки и целевой признак</a></span></li><li><span><a href="#Стоп-слова-и-векторизация" data-toc-modified-id="Стоп-слова-и-векторизация-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Стоп-слова и векторизация</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>CatBoost</a></span></li><li><span><a href="#Случайная-модель" data-toc-modified-id="Случайная-модель-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Случайная модель</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Импорт библиотек

In [2]:
!pip install catboost
!pip install tqdm
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import nltk
nltk.download('wordnet')
import re 
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import TfidfVectorizer 
from nltk.corpus import stopwords 
%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import catboost as cb
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Загрузка данных

In [2]:
toxic_comments = pd.read_csv('H:/toxic_comments.csv')
display(toxic_comments.head())
toxic_comments = toxic_comments.drop_duplicates()
toxic_comments.info()        

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.7+ MB


<div class="alert alert-info">
    
<font size="3"><b> 159571 строк без пропусков. Необходимо будет перевести в корпус текстов, очистить и лемматизировать для
    построения модели, где целевым признаком является столбец toxic.

</b></font>
    
</div>

###  Очистка и лемматизация

In [3]:
# корпус, текст в Юникод
corpus = toxic_comments['text'].values.astype('U')

# класс для лемматизации
WNL = WordNetLemmatizer()

# функция очистки и лемматизации 
def clean(text):
    
    text = text.lower()
    lemm_text = "".join(WNL.lemmatize(text))
    clear_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
    return " ".join(clear_text.split())

# обработка корпуса
for i in tqdm(range(len(corpus))):
    corpus[i] = clean(corpus[i])
# переводим из массива в таблицу
toxic_comments_corpus = pd.DataFrame(corpus)
toxic_comments_corpus.head()

100%|███████████████████████████████████████████████████████████████████████| 159571/159571 [00:14<00:00, 10713.20it/s]


,0
0,explanation why the edits made under my userna...
1,d aww he matches this background colour i m se...
2,hey man i m really not trying to edit war it s...
3,more i can t make any real suggestions on impr...
4,you sir are my hero any chance you remember wh...


In [4]:
# соединяем в общую таблицу
toxic_comments['lemm_text'] = toxic_comments_corpus[0]
display(toxic_comments.head())
toxic_comments.info()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he matches this background colour i m se...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it s...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


<class 'pandas.core.frame.DataFrame'>
Int64Index: 159571 entries, 0 to 159570
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   text       159571 non-null  object
 1   toxic      159571 non-null  int64 
 2   lemm_text  159571 non-null  object
dtypes: int64(1), object(2)
memory usage: 4.9+ MB


<div class="alert alert-info">
    
<font size="3"><b> Провели очистку и лемматизацию текста.

</b></font>
    
</div>

## Обучение

### Баланс классов

In [5]:
print(toxic_comments['toxic'].value_counts())

0    143346
1     16225
Name: toxic, dtype: int64


<div class="alert alert-info">
    
<font size="3"><b> С положительным значением почти в 10 раз меньше. Дисбаланс. Укажен в гиперпараметрах моделей баланс классов.

</b></font>
    
</div>

### Признаки и целевой признак

In [6]:
x = toxic_comments['lemm_text']
y = toxic_comments['toxic']

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=12345, stratify = y)

### Стоп-слова и векторизация

In [7]:
# загружаем стоп-слова
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

# TFIDF
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
                
# векторизация
x_train_tf_idf = count_tf_idf.fit_transform(x_train)
x_test_tf_idf = count_tf_idf.transform(x_test)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vladimir.DESKTOP-
[nltk_data]     SUBKJ68\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
print(x_train_tf_idf.shape)
print(x_test_tf_idf.shape)
print(y_train.shape)
print(y_test.shape)

(119678, 142100)
(39893, 142100)
(119678,)
(39893,)


<div class="alert alert-info">
    
<font size="3"><b> Определили признаки, разделили выборку на обучающую и тестовую. Провели векторизацию.

</b></font>
    
</div>

### Логистическая регрессия

In [9]:
%%time

# модель
LR = LogisticRegression(fit_intercept=True, 
                      n_jobs=None,
                      class_weight='balanced',
                      solver='lbfgs',  
                      random_state=12345
                     )
# словарь
hyperparams = {'C': [0.1, 1, 10]}

# обучение
LR_grid = GridSearchCV(LR, hyperparams, scoring='f1', cv=3)
LR_grid.fit(x_train_tf_idf, y_train)

# F1 с помощью кросс-валидации
LR.fit(x_train_tf_idf, y_train)
LR_cv_score = cross_val_score(LR ,x_train_tf_idf, y_train, scoring='f1', cv=3).mean()
print('Среднее качество Логистической регрессии на кросс-валидации:', LR_cv_score)

Среднее качество Логистической регрессии на кросс-валидации: 0.7476875732454532
Wall time: 43.4 s


In [10]:
%%time

LR_params = LR_grid.best_params_
LR_score = LR_grid.score(x_train_tf_idf, y_train)
print('Лучшее значение гиперпараметров: ', LR_params)
print('Лучшее значение F1: ', LR_score)

Лучшее значение гиперпараметров:  {'C': 10}
Лучшее значение F1:  0.9220527256818959
Wall time: 59.8 ms


In [11]:
%%time
# проверка на тестовой
LR_test_model = LogisticRegression(fit_intercept=True,
                                class_weight='balanced',
                                random_state=12345,
                                solver='lbfgs',
                                C=LR_params['C']
                               )

LR_test_model.fit(x_train_tf_idf, y_train)
LR_test_model_predict = LR_test_model.predict(x_test_tf_idf)

LR_F1 = round(f1_score(y_test, LR_test_model_predict), 3) 
print('Метрика F1: ', LR_F1)

Метрика F1:  0.777
Wall time: 4.44 s


### Случайный лес

In [12]:
%%time

# модель
RFC = RandomForestClassifier(class_weight='balanced', n_jobs=-1)
#словарь 
RFC_parametrs = {'n_estimators': range(15, 25, 5),
                     'max_depth': range(1, 8, 2),
                     'min_samples_leaf': range(1, 5),
                     'min_samples_split': range(2, 8, 2)}

RFC_grid = GridSearchCV(RFC, RFC_parametrs, scoring='f1', cv=3)
RFC_grid.fit(x_train_tf_idf, y_train)

# F1 с помощью кросс-валидации
RFC.fit(x_train_tf_idf, y_train)
RFC_cv_score = cross_val_score(RFC, x_train_tf_idf, y_train, scoring='f1', cv=3).mean()
print('Среднее качество Случайного леса на кросс-валидации:', RFC_cv_score)


Среднее качество Случайного леса на кросс-валидации: 0.6270805394280781
Wall time: 27min 13s


In [13]:
%%time

RFC_params = RFC_grid.best_params_
RFC_score = RFC_grid.score(x_train_tf_idf, y_train)
print('Лучшее значение гиперпараметров: ', RFC_params)
print('Лучшее значение F1: ', RFC_score) 

Лучшее значение гиперпараметров:  {'max_depth': 3, 'min_samples_leaf': 3, 'min_samples_split': 6, 'n_estimators': 20}
Лучшее значение F1:  0.2605310936663227
Wall time: 358 ms


In [14]:
%%time
# проверка на тестовой
RFC_test_model = RandomForestClassifier(random_state=42, n_jobs=-1, class_weight='balanced',
                                     max_depth=RFC_params['max_depth'],
                                     min_samples_leaf = RFC_params['min_samples_leaf'],
                                     min_samples_split = RFC_params['min_samples_split'],
                                     n_estimators = RFC_params['n_estimators'])

RFC_test_model.fit(x_train_tf_idf, y_train)
RFC_test_model_predict = RFC_test_model.predict(x_test_tf_idf)

RFC_F1 = round(f1_score(y_test, RFC_test_model_predict), 3) 
print('Метрика F1: ', RFC_F1)

Метрика F1:  0.447
Wall time: 395 ms


### CatBoost

In [15]:
%%time

# модель 
CB = cb.CatBoostClassifier(class_weights=[1, 143346/16225], 
                           verbose=10, 
                           loss_function="Logloss",
                           iterations=200
                          )
#словарь
CB_parametrs = {'learning_rate': [0.02, 0.3],
                'depth': [4, 6, 8]
               }

CB_grid = GridSearchCV(CB, CB_parametrs, scoring='f1', cv=3)
CB_grid.fit(x_train_tf_idf, y_train, verbose=10)

0:	learn: 0.6847399	total: 923ms	remaining: 3m 3s
10:	learn: 0.6294951	total: 6.05s	remaining: 1m 43s
20:	learn: 0.6018560	total: 11.1s	remaining: 1m 34s
30:	learn: 0.5837055	total: 16.3s	remaining: 1m 28s
40:	learn: 0.5686616	total: 21.3s	remaining: 1m 22s
50:	learn: 0.5563110	total: 26.2s	remaining: 1m 16s
60:	learn: 0.5454715	total: 31s	remaining: 1m 10s
70:	learn: 0.5371627	total: 35.9s	remaining: 1m 5s
80:	learn: 0.5286974	total: 40.7s	remaining: 59.8s
90:	learn: 0.5211454	total: 45.6s	remaining: 54.6s
100:	learn: 0.5146935	total: 50.4s	remaining: 49.4s
110:	learn: 0.5092538	total: 55.2s	remaining: 44.3s
120:	learn: 0.5036169	total: 1m	remaining: 39.2s
130:	learn: 0.4973420	total: 1m 4s	remaining: 34.2s
140:	learn: 0.4917916	total: 1m 9s	remaining: 29.1s
150:	learn: 0.4870895	total: 1m 14s	remaining: 24.2s
160:	learn: 0.4821851	total: 1m 19s	remaining: 19.2s
170:	learn: 0.4784299	total: 1m 24s	remaining: 14.3s
180:	learn: 0.4741990	total: 1m 28s	remaining: 9.34s
190:	learn: 0.4705

GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostClassifier object at 0x00000270059DE100>,
             param_grid={'depth': [4, 6, 8], 'learning_rate': [0.02, 0.3]},
             scoring='f1')

In [16]:
%%time

CB_params = CB_grid.best_params_
CB_score = CB_grid.score(x_train_tf_idf, y_train)
print('Лучшее значение гиперпараметров: ', CB_params)
print('Лучшее значение F1: ', CB_score)

Лучшее значение гиперпараметров:  {'depth': 6, 'learning_rate': 0.3}
Лучшее значение F1:  0.8006453268800645
Wall time: 951 ms


In [17]:
%%time

CB_test_model = cb.CatBoostClassifier(class_weights=[1, 143346/16225], 
                                      depth=CB_params['depth'],
                                      learning_rate=CB_params['learning_rate'],
                                      iterations=200)
CB_test_model.fit(x_train_tf_idf, y_train, verbose=10)
CB_model_predict = CB_test_model.predict(x_test_tf_idf)

CB_F1 = round(f1_score(y_test, CB_model_predict), 3)
print('Метрика F1: ', CB_F1)

0:	learn: 0.5963400	total: 1.54s	remaining: 5m 7s
10:	learn: 0.4410746	total: 15.9s	remaining: 4m 32s
20:	learn: 0.4008971	total: 29.9s	remaining: 4m 14s
30:	learn: 0.3730711	total: 43.8s	remaining: 3m 58s
40:	learn: 0.3491721	total: 57.9s	remaining: 3m 44s
50:	learn: 0.3301680	total: 1m 11s	remaining: 3m 29s
60:	learn: 0.3152757	total: 1m 25s	remaining: 3m 15s
70:	learn: 0.3021045	total: 1m 40s	remaining: 3m 2s
80:	learn: 0.2913827	total: 1m 54s	remaining: 2m 48s
90:	learn: 0.2824482	total: 2m 8s	remaining: 2m 33s
100:	learn: 0.2737554	total: 2m 22s	remaining: 2m 19s
110:	learn: 0.2664327	total: 2m 35s	remaining: 2m 5s
120:	learn: 0.2594647	total: 2m 49s	remaining: 1m 50s
130:	learn: 0.2536144	total: 3m 3s	remaining: 1m 36s
140:	learn: 0.2486191	total: 3m 17s	remaining: 1m 22s
150:	learn: 0.2435912	total: 3m 31s	remaining: 1m 8s
160:	learn: 0.2390864	total: 3m 45s	remaining: 54.5s
170:	learn: 0.2340574	total: 3m 58s	remaining: 40.5s
180:	learn: 0.2291962	total: 4m 12s	remaining: 26.5s

### Случайная модель

In [18]:
%%time

# модель
DUM_model = [1 for i in range(len(y_test))]

# предсказание
DUM_F1 = f1_score(y_test.reset_index(drop=True), DUM_model)

print('Метрика F1: ', DUM_F1)

Метрика F1:  0.18457757855696372
Wall time: 31.1 ms


In [3]:
# сводная таблица
model_F1=[
    ['Логистическая регрессия', '43.4 c', '0.777'],
    ['Случайный лес', '27 мин 13 с','0.447'],
    ['CatBoost', '38 мин 29 c', '0.755'],
    ['Cлучайная модель', '31.1 мc','0.185']
]
columns=['Model', 'Wall_time_fit', 'F1']
results_F1 = pd.DataFrame(data=model_F1, columns=columns)
display(results_F1) 

,Model,Wall_time_fit,F1
0,Логистическая регрессия,43.4 c,0.777
1,Случайный лес,27 мин 13 с,0.447
2,CatBoost,38 мин 29 c,0.755
3,Cлучайная модель,31.1 мc,0.185


<div class="alert alert-info">
    
<font size="3"><b> Обучили модели с учетом дисбаланса классов и проверили их на тестовой выборке.

</b></font>
    
</div>

## Выводы

<div class="alert alert-info">
    
<font size="3"><b> По результатам прошли две модели - Логистическая регрессия и CatBoost. Наилучший результат показала Логистическая регрессия, наихудший - Случайный лес. 
По времени также Логистическая регрессия наилучшая, время подбора параметров меньше минуты. 

</b></font>
    
</div>